In [3]:
! pip install pandas

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.4-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached numpy-2.2.4-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [6]:
import os 
from dotenv import load_dotenv

load_dotenv("C:/Users/user/Desktop/Finance-Model/.env")


True

In [34]:
! pip install python_dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [22]:
import pandas as pd 

def extract_csv_elements(data_path):
    """_summary_

    Args:
        data_path (_str__): csv 경로 
    """
    data = pd.read_csv(data_path, encoding="cp949", sep="\t")
    return data 


In [11]:
data = extract_csv_elements("../data/2024-2025-mirae_asset.csv")

print(data['상세내용'][0])

NameError: name 'extract_csv_elements' is not defined

In [44]:
from pydantic import BaseModel
from openai import OpenAI
import json
import os

# 1. OpenAI client 초기화
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")


# 2. CoT 데이터셋 Pydantic 모델 정의
class CoTDataset(BaseModel):
    instruction: str
    question: str
    complex_cot: str
    answer : str 



# 1. OpenAI client 초기화
client = OpenAI()

# 2. CoT 데이터셋 Pydantic 모델 정의 (키 변경)
class CoTDataset(BaseModel):
    instruction: str
    question: str
    complex_cot: str
    answer: str

# 3. 프롬프트 (새로운 키 이름 반영)
prompt = """
Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate exactly **{num_questions}** question(s), answer(s), and chain-of-thought reasoning.

You are a Teacher/Professor in {domain}. 
Your task is to create exactly {num_questions} diverse question(s) for an upcoming quiz/examination.
The question(s) should fully reflect your understanding of the context.
You must provide:
- A question extracted from the context.
- A detailed chain-of-thought (complex_cot) explaining your reasoning process.".
- A final answer in a complete sentence (in Korean)".

IMPORTANT:
- Return the result in JSON format containing the keys `instruction`, `question`, `complex_cot`, and `answer`.
  - `instruction` is fixed: "다음 Context를 바탕으로 문제 해결 과정을 거쳐 정답을 도출하세요."
  - `question` must contain the question text.
  - `complex_cot` must include the full reasoning process.
  - `answer` must include only the final answer".
- DO NOT use List or Array in JSON. Each QA pair must be a separate JSON object.

## Example Output:
```json
{{
    "instruction": "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.",
    "question": "배출권을 보유한 기업들이 가격 상승 시 판매를 고려할 수 있나요?",
    "detailed_explanation": "기업은 감축 노력을 통해 초과 배출권을 보유한 상태이며, 배출권 가격 상승은 보유 배출권의 단가를 높여 추가 수익 창출 가능성을 제공한다.",
    "answer": "정답: 배출권을 보유한 기업들은 가격이 상승할 경우 초과 배출권을 판매하여 수익을 창출할 수 있습니다. 예를 들어, 배출권 가격이 톤당 30,000원에서 50,000원으로 상승하면 100톤의 초과 배출권을 보유한 기업은 약 5,000만 원의 수익을 올릴 수 있습니다."
}}
"""



# 4. 데이터 생성 함수 정의
def generate_cot_dataset(context: str, domain: str = "AI", num_questions: int = 3):
    filled_prompt = prompt.format(context=context, domain=domain, num_questions=num_questions)

    # OpenAI structured output (Pydantic 기반)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating Chain of Thought (CoT) datasets."},
            {"role": "user", "content": filled_prompt}
        ],
        response_format=CoTDataset  # Pydantic 기반 응답 파싱
    )

    # 여러 개가 올 수 있으므로 choices로 받음
    structured_outputs = [choice.message.parsed for choice in completion.choices]
    return structured_outputs



In [163]:
! pip install openai==0.28

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-win_amd64.whl.metadata (36 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.3.0-cp310-cp310-win_amd64.whl.metadata (10 kB)
  Using cached yarl-1.18.3-cp310-cp310-win_amd64.whl.metadata (71 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)
Using cached charset_normalizer-3.4.1-cp310-cp310-win_amd64.whl (102 kB)
Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl (51 kB)
Using cached multidict-6.1.0-cp310-cp310-win

In [9]:
! pip install datasets

In [11]:
from datasets import Dataset, load_dataset

dataset = load_dataset("BCCard/BCCard-Finance-Kor-QnA", split="train")

data = dataset.to_pandas(dataset)
refine_data = data.loc[~data["instruction"].str.contains("비씨카드")].reset_index(drop=True)
dataset = Dataset.from_pandas(refine_data)

def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # output이 리스트일 경우 join()을 사용하여 하나의 문자열로 합칩니다.
        if isinstance(output, list):
            output_text = " ".join(output)
        else:
            output_text = output
        text = instruction + " " + output_text  # 문자열로 합칩니다.
        texts.append(text)
    return {"text": texts}

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)


Map: 100%|██████████| 30335/30335 [00:00<00:00, 80344.11 examples/s] 


In [36]:
dataset['text'][30000]

'스트림플레이션은 왜 최근 들어 이슈가 되고 있나요? 최근 몇 년간 OTT 서비스의 수요가 폭발적으로 증가하면서 다양한 플랫폼이 등장하고, 경쟁도 더욱 치열해졌습니다. 이에 따라 플랫폼 간 차별화된 콘텐츠 확보 경쟁이 격화되었고, 이를 위해 제작비와 운영 비용이 증가하면서 스트림플레이션 현상이 가속화되었습니다. 또한 코로나19 팬데믹 동안 많은 소비자가 OTT 서비스를 통해 엔터테인먼트를 소비하면서 스트리밍 시장이 성장했지만, 이후 구독료가 점차 인상되면서 소비자 부담이 늘어나고 있습니다. 현재 국내 OTT 시장은 약 7천억 원 규모로 추정되며, 2025년까지 약 1조 원에 이를 것으로 예상됩니다. 이에 따라 OTT 구독 비용에 대한 소비자의 민감도와 스트림플레이션에 대한 관심이 더욱 높아지고 있습니다.'

In [45]:
tmp = dataset["text"][30000]
results = generate_cot_dataset(tmp, domain="경제", num_questions=1)

In [46]:
tmp

'스트림플레이션은 왜 최근 들어 이슈가 되고 있나요? 최근 몇 년간 OTT 서비스의 수요가 폭발적으로 증가하면서 다양한 플랫폼이 등장하고, 경쟁도 더욱 치열해졌습니다. 이에 따라 플랫폼 간 차별화된 콘텐츠 확보 경쟁이 격화되었고, 이를 위해 제작비와 운영 비용이 증가하면서 스트림플레이션 현상이 가속화되었습니다. 또한 코로나19 팬데믹 동안 많은 소비자가 OTT 서비스를 통해 엔터테인먼트를 소비하면서 스트리밍 시장이 성장했지만, 이후 구독료가 점차 인상되면서 소비자 부담이 늘어나고 있습니다. 현재 국내 OTT 시장은 약 7천억 원 규모로 추정되며, 2025년까지 약 1조 원에 이를 것으로 예상됩니다. 이에 따라 OTT 구독 비용에 대한 소비자의 민감도와 스트림플레이션에 대한 관심이 더욱 높아지고 있습니다.'

In [53]:
save_path = "../data/finance_CoT.json"
def save_to_json(data, filename=save_path):
    if not data:
        print("⚠️ [WARNING] 저장할 데이터가 없습니다!")
        return  # 데이터가 없으면 저장하지 않음

    os.makedirs(os.path.dirname(filename), exist_ok=True)  # ✅ 경로가 없으면 생성

    # ✅ 기존 파일 로드
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            try:
                existing_data = json.load(f)
            except json.JSONDecodeError:
                print("⚠️ JSON 파일이 손상되었습니다. 새로 생성합니다.")
                existing_data = []
    else:
        existing_data = []

    # ✅ CoTDataset 객체를 `dict`로 변환 후 저장
    converted_data = [item.dict() for item in data]  # ✅ 여기서 `dict()` 변환 추가
    
    existing_data.extend(converted_data)

    # ✅ JSON 파일 저장
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)
    print(f"✅ [INFO] 데이터 저장 완료! (총 {len(existing_data)}개)")

In [54]:
save_to_json(results)

⚠️ JSON 파일이 손상되었습니다. 새로 생성합니다.
✅ [INFO] 데이터 저장 완료! (총 1개)


C:\Users\user\AppData\Local\Temp\ipykernel_1864\93681718.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  converted_data = [item.dict() for item in data]  # ✅ 여기서 `dict()` 변환 추가


In [8]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {
            "role": "user",
            "content": "3.9와 3.11중에 어떤 값이 더 크니? 왜 그런지 설명도 해봐"
        }
    ]
)

print(completion.choices[0].message.content)

3.11이 3.9보다 더 큽니다. 

이유는 소수점 이하의 자리수를 비교했을 때, 3.11의 첫 번째 소수점 이하 숫자인 1이 3.9의 첫 번째 소수점 이하 숫자인 9보다 작기 때문입니다. 따라서 3.11은 3.1(3와 1에 해당하는 부분)과 0.01(0.1과 0.01에 해당하는 부분)을 포함하고, 3.9는 3.9(3와 9에 해당하는 부분)로 더 큰 수가 아닙니다. 

소수를 비교할 때는 소수점 이하의 자리수를 차례대로 비교해 나가는 방식으로 진행하므로, 최종적으로 3.11이 3.9보다 더 크다는 결론을 내리게 됩니다.


In [27]:

prompt_template = """
Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate exactly **{num_questions}** question(s), answer(s), and a detailed explanation.

You are a Teacher/Professor in {domain}. 
Your task is to create exactly {num_questions} diverse question(s) for an upcoming quiz/examination.
The question(s) should fully reflect your understanding of the context.
You must provide:
- A question extracted from the context.
- A detailed explanation (provide reasoning, step by step, but do not reveal chain-of-thought in a raw format).
- A final answer in a complete sentence (in Korean), prefixed with "정답:".

IMPORTANT:
- You MUST only provide factual and accurate information based on the provided context. Do NOT fabricate or assume any information that is not clearly stated in the context.
- You MUST mention the company name '{company_name}' explicitly in both 'question', 'detailed_explanation', and 'answer'. Do NOT use vague expressions like 'the company' or '동사'. You must directly write '{company_name}' when referring to the company.
- Return the result in JSON format containing the keys `instruction`, `question`, `detailed_explanation`, and `answer`.
  - `instruction` is fixed: "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요."
  - `question` must contain the question text.
  - detailed_explanation: Based on the provided context, include specific numerical values related to {company_name} (e.g., revenue, EPS, growth rate, RPO, QoQ, etc.) and explain in detail the logical thought process on how these figures contributed to deriving the final answer.
  - answer: Include only the final answer, ensuring that {company_name} is mentioned, and present it concisely.
- DO NOT use List or Array in JSON. Each QA pair must be a separate JSON object.

## Example Output:
```json
{{
    "instruction": "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.",
    "question": "FY4Q24 기간 동안 스타벅스 주가가 3개월 간 30.4% 상승했음에도 불구하고, 이 주가 상승이 악화된 분기 실적과 조화되지 않는 이유는 무엇인가?",
    "detailed_explanation": "FY4Q24 기간 동안 스타벅스의 주요 재무 지표 중에서 '영업이익', '북미 CSSG', '영업이익률 - 인터내셔널' 등이 AI 모델의 예상치를 하회하면서 실적이 전반적으로 악화되었습니다. 그럼에도 불구하고 스타벅스의 주가는 최근 3개월간 오히려 30.4% 상승하는 모습을 보였습니다.
이러한 모순이 발생한 첫 번째 이유는 현재 스타벅스의 PER(주가수익비율)이 26.3배로, 과거 평균 PER인 24.1배 대비 높은 수준에서 거래되고 있기 때문입니다. 이는 시장 참여자들이 최근 악화된 분기 실적보다는 스타벅스가 발표한 새로운 전략('스타벅스로 돌아가기') 및 향후 실적 개선 가능성 등 미래 성장에 대한 기대감을 더 강하게 반영하고 있음을 시사합니다.
두 번째 이유는 최근 스타벅스의 주가 움직임이 시장 전체 지수와의 상관관계가 약화된 상태에서 이루어졌다는 점입니다. 일반적으로 특정 주식의 주가가 시장 지수와 상관관계가 낮아질 경우, 해당 주식의 가격은 전체 시장 흐름보다는 개별기업 이슈나 기업 고유의 기대감과 같은 내부적 혹은 특정 투자자들의 심리적 요인의 영향을 더 크게 받을 가능성이 높아집니다.
이 두 가지 측면을 종합하면, 스타벅스의 현재 주가 상승은 최근의 부진한 재무 실적이 반영되지 않고, 오히려 시장의 미래 기대치와 개별적 투자 요인들로 인해 주가가 높게 형성된 것으로 분석할 수 있습니다.",
    "answer": "스타벅스 주가 상승은 과거 대비 높은 PER 수준에서 나타난 시장의 미래 성장 기대감과, 시장 지수와의 상관관계 약화로 인해 실적 악화가 주가에 적절히 반영되지 않았기 때문입니다."
}}
"""



def main(data: str, idx: int):
    # 3) 필요한 변수 세팅
    context = data["상세내용"][idx]
    domain = "경제"
    num_questions = 1

    # 실제 질문

    # 4) prompt_template.format(...)로 지시사항을 채우고, 뒤에 실제 질문을 연결
    user_content = (
        prompt_template.format(context=context, domain=domain, num_questions=num_questions)
    )

    # 5) ChatCompletion 호출 (system 메시지 없음, user 메시지만 사용)
    response = client.chat.completions.create(
        model="o1-mini-2024-09-12",
        messages=[
            {
                "role": "user",
                "content": user_content
            }
        ]
    )

    # 6) 결과 출력
    return response.choices[0].message.content

In [232]:
data_lst

['```json\n{\n    "instruction": "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.",\n    "question": "경제학에서 \'더\'의 의미는 무엇인가?",\n    "detailed_explanation": "주어진 컨텍스트는 \'더\'라는 단어 하나뿐입니다. 한국어에서 \'더\'는 \'더 많이\', \'추가적으로\', \'더욱\' 등의 의미로 사용되며, 경제학에서는 일반적으로 추가적인 양이나 증가를 나타내는 부사로 활용됩니다. 예를 들어, 기업의 생산량을 \'더\' 증가시키거나 소비자의 지출을 \'더\' 유도하는 상황에서 사용될 수 있습니다.",\n    "answer": "정답: \'더\'는 경제학에서 추가적인 양이나 증가를 나타내는 의미로 사용됩니다."\n}\n```',
 '```json\n{\n    "instruction": "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.",\n    "question": "\'기\'라는 단어가 경제학에서 사용될 때 어떤 의미를 가질 수 있는지 설명하시오.",\n    "detailed_explanation": "주어진 컨텍스트는 단어 \'기\' 하나로 제한되어 있습니다. \'기\'는 한국어에서 다양한 의미를 가질 수 있으며, 경제학에서는 주로 \'기초\', \'기술\', \'기금\' 등으로 사용될 수 있습니다. 예를 들어, \'기초 자본\'은 사업을 시작하는 데 필요한 기본 자본을 의미할 수 있고, \'기술 혁신\'은 경제 성장에 중요한 요소로 작용할 수 있습니다. 또한, \'기금\'은 특정 목적을 위해 마련된 재원을 의미하기도 합니다. 따라서 \'기\'는 문맥에 따라 다양한 경제적 개념을 나타낼 수 있습니다.",\n    "answer": "정답: \'기\'는 경제학에서 문맥에 따라 \'기초\', \'기술\', \'기금\' 등 다양한 의미로 사용될 수 있습니다."\n}\n```',
 

In [13]:
import pandas as pd 
data = pd.read_csv("../data/2024-2025-mirae_asset.csv", encoding="cp949", sep="\t")

In [31]:
tmps = data.iloc[1003:1004]

In [32]:
tmps

,Unnamed: 0,회사명,번호,매수의견,view1,view2,압축설명,상세URL,상세내용
1003,1003,엔비디아,NVDA US,Not Rated,2323319,199,1Q 실적 Quick review,https://securities.miraeasset.com/bbs/board/me...,1Q 실적 Quick review주가는 우선 Blackwell 자체에 환호 동사의 ...


In [30]:
data["상세내용"][1003]

'1Q 실적 Quick review주가는 우선 Blackwell 자체에 환호 동사의 1분기 실적은 매출액 $26.0B(YoY +262%, QoQ +17.8%), GPM 78.9%를 기록하며 시장 기대치를 상회했다. Gaming 및 ProVis 사업부의 QoQ -7 ~ 8%대 감소에도 불구하고 데이터센터향 매출액이 QoQ +22.6% 증가하며 전사 실적을 견인했다. 한편 HGX보드와 GPU에 치중되어 있던 수요가 GB200 시스템을 비롯한 시스템 사업으로 이동하는 추세가 확인되며, 이는 100가지가 넘는 컴퓨터 시스템 구성을 보유한 Blackwell 플랫폼을 기반으로 확장된 제품 포트폴리오에 기인한다. 클라우드와 기업고객 모두에게서 수요가 강세였다. 테슬라의 Training AI cluster는 H100 GPU 35,000개 규모로 확장되었고, 메타의 LLM인 Llama 3는 24,000개의 H100 GPU로 학습되었다. H100에 비해 추론성능이 2배 가까이 개선된 H200은 1Q에 샘플링을 시작했으며 2Q 양산을 계획대로 진행중이다. 첫번째 H200 제품은 OpenAI에 공급되었으며 이를 기반으로 GPT-4o의 데모가 이뤄졌다. 차기 플랫폼인 Blackwell의 경우 2Q(5 ~ 7월)내 초도물량 출하 예정이며, 3Q내 양산 개시, 4Q에 데이터센터에 탑재될 것으로 예상했다. 초도물량 출하 시기가 시장 기대(2Q) 보다 이르게 전개되는 모습이다. Blackwell은 H100 대비 4배의 학습속도와 30배의 추론속도를 가능케 하며 25배의 유지비용 절감을 가능케 한다. 2분기 실적은 매출액 $28.0B(YoY +107%, QoQ +7.5%), GPM 75.5%로 시장 기대치를 상회했다. 수익성은 임계점에 다다른 것으로 판단되나, Hopper 수요 강세, H200/Blackwell 전환 등 수요과잉 상황에 매출액 성장은 지속될 것으로 전망된다. 금년도 연간 GPM은 70% 중반대 범위로 제시했다. 단기적 관점에서는 B100, GB200 나아가 X100 등 프로세

In [21]:
tmp = main(data, 497)

KeyError: 'company_name'

In [220]:
json.loads(tmp[8:-4])

{'instruction': '다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.',
 'question': 'Firefly가 주도하는 AI 제품의 현재 ARR은 얼마이며, FY25까지의 목표 ARR은 얼마입니까? 또한, 이를 달성하기 위한 주요 성장 동력은 무엇인가요?',
 'detailed_explanation': '컨텍스트에 따르면 Firefly의 AI 제품인 Acrobat AI Assistant, Firefly, GenStudio 등의 ARR은 현재 1.25억 달러입니다. 기업은 FY25까지 이 ARR을 2배 이상으로 확대하려는 목표를 설정하였습니다. 이를 달성하기 위한 주요 성장 동력으로는 고객들이 Firefly의 AI 기능을 활용하기 위해 상위 플랜으로 업그레이드하는 AI 업셀링 증가와 Firefly Standard/Pro/Premium 요금제의 독립형 구독 모델 출시를 통한 신규 수익원 확보가 있습니다.',
 'answer': '정답: Firefly의 현재 ARR은 1.25억 달러이며, FY25까지 목표 ARR은 2.5억 달러입니다. 이를 달성하기 위한 주요 성장 동력은 AI 업셀링 증가와 독립형 구독 모델 출시입니다.'}

In [45]:
import os
from openai import OpenAI
import pandas as pd
import time

# OpenAI 클라이언트 설정
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")  

prompt_template = """
Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate exactly **{num_questions}** question(s), answer(s), and a detailed explanation.

You are a Teacher/Professor in {domain}. 
Your task is to create exactly {num_questions} diverse question(s) for an upcoming quiz/examination.
The question(s) should fully reflect your understanding of the context.
You must provide:
- A question extracted from the context.
- A detailed explanation (provide reasoning, step by step, but do not reveal chain-of-thought in a raw format).
- A final answer in a complete sentence (in Korean), prefixed with "정답:".

IMPORTANT:
- You MUST only provide factual and accurate information based on the provided context. Do NOT fabricate or assume any information that is not clearly stated in the context.
- You MUST mention the company name '{company_name}' explicitly in both 'question', 'detailed_explanation', and 'answer'. Do NOT use vague expressions like 'the company' or '동사'. You must directly write '{company_name}' when referring to the company.
- Return the result in JSON format containing the keys `instruction`, `question`, `detailed_explanation`, and `answer`.
  - `instruction` is fixed: "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요."
  - `question` must contain the question text.
  - detailed_explanation: Based on the provided context, include specific numerical values related to {company_name} (e.g., revenue, EPS, growth rate, RPO, QoQ, etc.) and explain in detail the logical thought process on how these figures contributed to deriving the final answer.
  - answer: Include only the final answer, ensuring that {company_name} is mentioned, and present it concisely.
- DO NOT use List or Array in JSON. Each QA pair must be a separate JSON object.

## Example Output:
```json
{{
    "instruction": "다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.",
    "question": "FY4Q24 기간 동안 스타벅스 주가가 3개월 간 30.4% 상승했음에도 불구하고, 이 주가 상승이 악화된 분기 실적과 조화되지 않는 이유는 무엇인가?",
    "detailed_explanation": "FY4Q24 기간 동안 스타벅스의 주요 재무 지표 중에서 '영업이익', '북미 CSSG', '영업이익률 - 인터내셔널' 등이 AI 모델의 예상치를 하회하면서 실적이 전반적으로 악화되었습니다. 그럼에도 불구하고 스타벅스의 주가는 최근 3개월간 오히려 30.4% 상승하는 모습을 보였습니다.
이러한 모순이 발생한 첫 번째 이유는 현재 스타벅스의 PER(주가수익비율)이 26.3배로, 과거 평균 PER인 24.1배 대비 높은 수준에서 거래되고 있기 때문입니다. 이는 시장 참여자들이 최근 악화된 분기 실적보다는 스타벅스가 발표한 새로운 전략('스타벅스로 돌아가기') 및 향후 실적 개선 가능성 등 미래 성장에 대한 기대감을 더 강하게 반영하고 있음을 시사합니다.
두 번째 이유는 최근 스타벅스의 주가 움직임이 시장 전체 지수와의 상관관계가 약화된 상태에서 이루어졌다는 점입니다. 일반적으로 특정 주식의 주가가 시장 지수와 상관관계가 낮아질 경우, 해당 주식의 가격은 전체 시장 흐름보다는 개별기업 이슈나 기업 고유의 기대감과 같은 내부적 혹은 특정 투자자들의 심리적 요인의 영향을 더 크게 받을 가능성이 높아집니다.
이 두 가지 측면을 종합하면, 스타벅스의 현재 주가 상승은 최근의 부진한 재무 실적이 반영되지 않고, 오히려 시장의 미래 기대치와 개별적 투자 요인들로 인해 주가가 높게 형성된 것으로 분석할 수 있습니다.",
    "answer": "스타벅스 주가 상승은 과거 대비 높은 PER 수준에서 나타난 시장의 미래 성장 기대감과, 시장 지수와의 상관관계 약화로 인해 실적 악화가 주가에 적절히 반영되지 않았기 때문입니다."
}}
"""

# 1) 데이터 한 건에 대해 OpenAI 호출하는 함수
def call_openai_api(context: str, company_name: str, domain: str = "경제", num_questions: int = 1):
    # 프롬프트 채우기
    user_content = prompt_template.format(
        context=context,
        domain=domain,
        num_questions=num_questions,
        company_name=company_name  # 회사명을 명시적으로 넣어줌
    )

    # 2. API 호출
    response = client.chat.completions.create(
        model="gpt-4o-2024-11-20",
        messages=[
            {"role": "user", "content": user_content}
        ],
    )

    # 3. 결과 반환
    return response.choices[0].message.content


# 2) 전체 데이터프레임을 순회하면서 API 호출
def process_data(df: pd.DataFrame, context_column: str = "상세내용", company_column: str = "회사명"):
    results = []
    
    for idx, row in df.iloc[:5].iterrows():
        context = row[context_column]
        company_name = row[company_column]  # 회사명 추가로 받아오기
        try:
            print(f"[INFO] Processing index {idx}, Company: {company_name}...")
            result = json.loads(call_openai_api(context=context, company_name=company_name)[8:-4])
            results.append({
                "index": idx,
                "company_name": company_name,
                "context": context,
                "result": result
            })
            time.sleep(1)  # API 요청 간 간격
        except Exception as e:
            print(f"[ERROR] Failed to process index {idx}: {e}")
            results.append({
                "index": idx,
                "company_name": company_name,
                "context": context,
                "result": None,
                "error": str(e)
            })

    return results


In [46]:
import json

processed_results = process_data(data, context_column="상세내용")

# 결과 확인
for res in processed_results:
    print(f"\n=== Index {res['index']} ===")
    print(res["result"])

[INFO] Processing index 0, Company: 어도비...
[INFO] Processing index 1, Company: LG에너지솔루션...
[INFO] Processing index 2, Company: 오라클...
[INFO] Processing index 3, Company: [월간 리서치 - 증권] 키움증권...
[INFO] Processing index 4, Company: [월간 리서치 - 증권]  NH투자증권...

=== Index 0 ===
{'instruction': '다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.', 'question': '어도비의 FY2Q25 가이던스에서 제시된 매출 전망과 Non-GAAP EPS가 투자자들에게 아쉬움을 남긴 이유는 무엇인가?', 'detailed_explanation': '어도비는 FY2Q25 가이던스에서 매출을 57.7-58.2억 달러로, Non-GAAP EPS를 4.95-5달러로 예상했습니다. 이 수치는 시장 예상치였던 매출 58억 달러와 EPS 5달러를 모두 소폭 하회하는 결과를 나타냅니다. 매출 전망의 경우, 하단인 57.7억 달러는 시장 예상 대비 낮고, 상단도 58억 달러와 일치하지만 더 나은 기대치를 충족하지 못했습니다. 마찬가지로 Non-GAAP EPS도 4.95-5달러로 제시했는데, 이는 시장 기대치 최소값인 5달러와 같은 수준 또는 그 이하로 나타나 시장의 기대를 낮추는 영향을 미쳤습니다. 이러한 가이던스는 어도비가 지속적인 성장 동력은 갖추고 있으나, 현재 시장 내 경쟁 심화와 AI 수익화 속도의 완만함 등이 기대보다 더 긍정적이지 않은 요인으로 작용했을 가능성을 시사합니다. 가이던스 수치가 실적 기대치를 상회하지 못한 점이 투자자들에게 아쉬움을 남긴 핵심 이유가 됩니다.', 'answer': '정답: 어도비의 FY2Q25 가이던스에서 매출 57.7-58.2억 달러와 Non-GAAP EPS 4.95-5달러는 시장 예

In [47]:
for i in processed_results:
    for j,k in i["result"].items():
        print(f"{j}: {k}")
        

instruction: 다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.
question: 어도비의 FY2Q25 가이던스에서 제시된 매출 전망과 Non-GAAP EPS가 투자자들에게 아쉬움을 남긴 이유는 무엇인가?
detailed_explanation: 어도비는 FY2Q25 가이던스에서 매출을 57.7-58.2억 달러로, Non-GAAP EPS를 4.95-5달러로 예상했습니다. 이 수치는 시장 예상치였던 매출 58억 달러와 EPS 5달러를 모두 소폭 하회하는 결과를 나타냅니다. 매출 전망의 경우, 하단인 57.7억 달러는 시장 예상 대비 낮고, 상단도 58억 달러와 일치하지만 더 나은 기대치를 충족하지 못했습니다. 마찬가지로 Non-GAAP EPS도 4.95-5달러로 제시했는데, 이는 시장 기대치 최소값인 5달러와 같은 수준 또는 그 이하로 나타나 시장의 기대를 낮추는 영향을 미쳤습니다. 이러한 가이던스는 어도비가 지속적인 성장 동력은 갖추고 있으나, 현재 시장 내 경쟁 심화와 AI 수익화 속도의 완만함 등이 기대보다 더 긍정적이지 않은 요인으로 작용했을 가능성을 시사합니다. 가이던스 수치가 실적 기대치를 상회하지 못한 점이 투자자들에게 아쉬움을 남긴 핵심 이유가 됩니다.
answer: 정답: 어도비의 FY2Q25 가이던스에서 매출 57.7-58.2억 달러와 Non-GAAP EPS 4.95-5달러는 시장 예상치(매출 58억 달러, EPS 5달러)를 소폭 하회하며 투자자들의 기대에 미치지 못했습니다.
instruction: 다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.
question: 2025년 1분기 LG에너지솔루션의 매출액 및 영업이익 추정치가 상향 조정된 주요 이유는 무엇인가?
detailed_explanation: Context에서 제시된 내용을 기반으로 LG에너지솔루션의 2025년 1분기 매출액과 영업이익 추정치가 각각 6.2조원 및 1,270억원으로 상향

In [44]:
print(f"{data['상세내용'][0]}\n\n")

더 확실한 한 방이 필요한 시점견조한 실적, 아쉬운 가이던스 FY1Q25 예상을 상회하는 실적을 발표했다. 매출 57.1억달러(+10% YoY, 예상치 56.6억달러 상회), Non-GAAP EPS 5.08달러(+13% YoY, 예상치 4.97달러 상회)를 기록했다. 그러나 가이던스가 예상을 하회했다. FY2Q25 가이던스로 매출 57.7-58.2억달러(예상치 58억달러 하회), Non-GAAP EPS 4.95-5달러(예상치 5달러 하회)를 제시했다. FY25 가이던스는 매출 233-235.5억달러, Non-GAAP EPS 20.2-20.5달러로, 예상치를 소폭 하회했다.Firefly가 주도한 AI 성과 Firefly의 성과는 조금씩 가시화되고 있다. Acrobat AI Assistant, Firefly, GenStudio 등 독립형 AI 제품의 ARR은 1.25억달러였다. FY25까지 해당 ARR을 2배 이상 확대하겠다는 목표를 제시했다. 고객들이 Firefly AI 기능을 활용하기 위해 상위 플랜으로 업그레이드하는 AI 업셀링 증가가 주된 성장 동력으로 작용했다. 또한, Firefly Standard/Pro/Premium 요금제의 독립형 구독 모델 출시를 통해 신규 수익원을 확보한 점도 긍정적이다. 현재 유료 사용자 중 90% 이상이 Firefly의 비디오 생성 기능을 활용 중이다. Firefly가 고객에게 실질적인 가치를 제공한다는 점에서 추가 구독률과 도입률 증가 가능성은 높다고 판단한다. 그러나 실적 기여도가 아직까지는 크지 않다. 현재까지의 AI 매출은 대부분 Creative Cloud 사업부에서 발생하고 있다. AI 성장 가속화를 위해 AI Assistant 등 Document Cloud 사업부에서의 AI 매출 기여도 확대가 필요한 상황이다.AI 수익화 속도가 관건 동사는 이번 실적 발표에서 현재까지의 AI 도입 속도와 수익화 가능성에 만족하고 있다고 코멘트했다. 그러나 현재 AI 관련 매출은 전체 연간 매출의 한 자릿수 비중에 불과하다. 다음주에

In [303]:
[i["result"] for i in processed_results][1]

{'instruction': '다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.',
 'question': 'LG에너지솔루션의 1분기 2025년 매출액과 영업이익 추정치가 상향 조정된 이유는 무엇인가?',
 'detailed_explanation': 'LG에너지솔루션의 1분기 2025년 매출액이 기존 예측보다 6% 상승한 6.2조 원으로, 영업이익 역시 기존의 영업손실 예상치에서 1,270억 원의 흑자로 변경되었습니다. 이러한 상향 조정의 주요 이유는 세 가지로 요약됩니다. 첫째, 3월에 중국 남경 공장에서 Tesla Model Y의 신규 모델 향 원통형 셀 출하가 시작되면서 배터리 출하량이 증가하였습니다. 둘째, 예상보다 견조한 유럽 전기차(EV) 수요, 특히 폭스바겐을 중심으로 한 수요 증가가 매출에 긍정적인 영향을 미쳤습니다. 셋째, 리튬, 니켈, 코발트 등 주요 메탈의 가격이 최근 하향 안정화되면서 원가 부담이 줄어들고, 판가 스프레드가 개선되어 영업이익이 상승하였습니다. 이러한 요소들이 종합적으로 작용하여 LG에너지솔루션의 1분기 실적이 기존 예상보다 긍정적으로 상향 조정되었습니다.',
 'answer': 'LG에너지솔루션의 1분기 2025년 매출액과 영업이익 상향 조정은 Tesla Model Y 신규 모델 출하 증가, 유럽 폭스바겐 중심의 견조한 EV 수요, 그리고 주요 메탈 가격 안정화로 인한 원가 절감이 주요 원인입니다.'}

## 결론

1. 

In [74]:
dataset = load_dataset("BCCard/BCCard-Finance-Kor-QnA", split="train")

save_path = "../data/finance_CoT.json"

data = dataset.to_pandas(dataset)

refine_data = data.loc[
    ~(data["instruction"].str.contains("카드|BC|비씨|비씨카드|BC카드|생명보험|라이프|보험|화재|최우형", regex=True)) & 
    ~(data["output"].str.contains("카드|BC|비씨|비씨카드|BC카드|생명보험|라이프|보험|화재|최우형", regex=True))
].reset_index(drop=True)

In [86]:
refine_data.iloc[78]

instruction         BNK경남 The Gallery Card의 TOP포인트 적립 조건은 무엇인가요?
output         TOP가맹점 이용 시 최대 10% TOP포인트가 적립됩니다. 생활서비스 제공 조건을...
Name: 78, dtype: object

In [82]:
refine_data.loc[refine_data["instruction"].str.contains("경제")]

,instruction,output
350,무이자 할부가 경제에 미치는 영향은 무엇인가요?,"무이자 할부는 소비를 촉진하여 단기적으로는 소매 매출을 늘릴 수 있으나, 과소비를 ..."
361,무이자 할부의 경제적 영향은 무엇인가요?,무이자 할부는 소비를 촉진하여 단기적으로는 경제 활동을 활성화시킬 수 있습니다. 그...
368,무이자 할부의 시장 경제적 효과는 어떤가요?,시장에서 무이자 할부는 소비자 구매력을 일시적으로 증가시켜 경제적 활동을 자극할 수...
906,수수료율 인하 정책이 지역 경제에 미치는 영향은 무엇인가요?,수수료율 인하 정책은 지역 경제에 긍정적인 영향을 미칩니다. 소상공인의 경영 부담이...
915,가맹점 수수료 인하가 지역 경제에 미치는 영향은 무엇인가요?,가맹점 수수료 인하는 지역 소상공인의 경제적 부담을 줄여줄 수 있어 지역 경제 활성...
...,...,...
18679,소비자물가지수를 통해 경제 불황을 예측할 수 있나요?,소비자물가지수(CPI)는 경제 불황의 신호를 감지하는 데 중요한 역할을 할 수 있습...
18691,환율 상승이 전반적인 경제 성장률에 어떤 영향을 미칠 수 있나요?,환율 상승은 경제 성장률에 복합적인 영향을 미칠 수 있습니다. 수출 산업이 강한 국...
18702,환율 변동이 국가 경제에 미치는 전체적인 영향은 무엇인가요?,환율 변동은 국가 경제에 복합적인 영향을 미칩니다. 환율 상승(원화 가치 하락)은 ...
18708,CPI 상승률이 경제 성장에 미치는 영향은 무엇인가요?,CPI 상승률은 경제 성장에 양면적인 영향을 미칠 수 있습니다. 적정한 물가 상승률...


In [76]:
refine_data.shape[0]

19043

In [90]:
with open("C:/Users/user/Desktop/Finance-Model/data/cleaned_data.json", 'r', encoding="utf-8") as f:
    data = json.load(f)

In [92]:
len(data)

15808

In [107]:
dataset = load_dataset("UICHEOL-HWANG/GreenFinance-Finance_CoT", split="train")

In [97]:
dataset['result']

[{'answer': '정답: 부킹 홀딩스의 2024년 연간 수익은 전년 대비 11% 증가했으며, 비용 관리를 통해 더 빠른 수익성 성장을 달성했습니다.',
  'detailed_explanation': '부킹 홀딩스는 2024년 실적 발표에서 연간 수익이 전년 대비 11% 증가했다고 밝혔습니다. 이는 CEO 글렌 포겔이 언급한 바에 따르면, 비용 관리를 통해 더 빠른 수익성 성장을 달성한 결과입니다. 구체적으로, 중개매출액과 광고 및 기타 매출액이 AI 예측치를 상회하면서 전체 수익 증가에 기여했지만, 총매출액과 판매매출마진율은 하회했습니다. 비용 관리의 효율성이 수익성 향상에 중요한 역할을 했다는 점이 성장을 이끈 주요 요인으로 작용했습니다.',
  'instruction': '다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을 도출하세요.',
  'question': '부킹 홀딩스의 2024년 연간 수익은 전년 대비 몇 퍼센트 증가했으며, 이러한 성장을 달성한 주요 요인은 무엇인가요?'},
 {'answer': "비야디의 '천신의 눈' 자율주행 시스템은 A, B, C 버전으로 나뉘며, 이 시스템은 약 1,385만원에서 4,958만원 사이의 다양한 가격대 차량에 탑재될 예정입니다.",
  'detailed_explanation': "비야디는 자율주행 시스템 '천신의 눈'을 자체 개발하여 21종 모델에 1차적으로 탑재할 계획입니다. 이 시스템은 10만, 15만, 20만 위안대의 모든 라인업에 탑재되며, 10만 위안 이하 모델에는 일부 탑재됩니다. '천신의 눈'은 A, B, C 3가지 버전으로 나뉘며, A 버전은 DiPilot 600 플랫폼(라이다+레이더+카메라)을 기반으로 럭셔리 브랜드 양왕 모델에 탑재됩니다. B 버전은 DiPilot 300 플랫폼(라이다+레이더+카메라 수 감소)을 기반으로 프리미엄 브랜드 덴자와 BYD 브랜드에 탑재되며, C 버전은 DiPilot 100 플랫폼(카메라+레이더)을 기반으로 BYD 브랜드에 탑재

In [110]:
import re 
# 전처리 함수
def clean_text(example):
    def process_text(text):
        if text is None:
            return ""  # None 값 방지
        text = re.sub(r'정답:\s*', '', text)  # "정답:" 제거
        text = re.sub(r'\[.*?\]', '', text).strip()  # 대괄호 안의 내용 제거
        return text

    # 모든 필드에 대해 처리 적용
    example["result"]["answer"] = process_text(example["result"]["answer"])
    example["result"]["detailed_explanation"] = process_text(example["result"]["detailed_explanation"])
    example["result"]["instruction"] = process_text(example["result"]["instruction"])
    example["result"]["question"] = process_text(example["result"]["question"])
    
    return example

# 전처리 적용
cleaned_dataset = dataset.map(clean_text)


Map: 100%|██████████| 917/917 [00:00<00:00, 5727.79 examples/s]


In [125]:
import pandas as pd
pd.set_option('display.max_columns', None)

data = pd.DataFrame(cleaned_dataset["result"])


In [126]:
data

,answer,detailed_explanation,instruction,question
0,"부킹 홀딩스의 2024년 연간 수익은 전년 대비 11% 증가했으며, 비용 관리를 통...",부킹 홀딩스는 2024년 실적 발표에서 연간 수익이 전년 대비 11% 증가했다고 밝...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,"부킹 홀딩스의 2024년 연간 수익은 전년 대비 몇 퍼센트 증가했으며, 이러한 성장..."
1,"비야디의 '천신의 눈' 자율주행 시스템은 A, B, C 버전으로 나뉘며, 이 시스템...",비야디는 자율주행 시스템 '천신의 눈'을 자체 개발하여 21종 모델에 1차적으로 탑...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,비야디가 발표한 자율주행 시스템 '천신의 눈'의 버전별 특징과 해당 시스템이 탑재될...
2,루닛 Insight는 Fuji Film 방향 매출의 71%와 동국생명과학 방향 매출...,"주어진 컨텍스트에서, Fuji Film 방향 매출은 총 26억원 중 해외 루닛 In...",다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,Volpara Health의 주요 고객사별 루닛 Insight 매출 비중은 어떻게 ...
3,KB금융은 2025년까지 주주환원율을 개선하기 위해 연간 위험가중자산 증가율을 4....,KB금융은 2025년까지 주주환원율을 개선하기 위해 두 가지 주요 방안을 설정했습니...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,KB금융이 2025년까지 주주환원율을 개선하기 위해 설정한 두 가지 주요 방안은 무...
4,에이엘티의 1분기 영업손실은 전방 산업 부진에 따른 매출 감소와 설비 증설로 인한 ...,"에이엘티는 1Q24 매출액이 90.1억원으로 전분기 대비 19.6% 감소하고, 전년...",다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,에이엘티의 1분기 영업손실이 발생한 주요 원인은 무엇인가요?
...,...,...,...,...
912,볼파라 인수를 통해 루닛의 3분기 매출은 전년 동기 대비 413% 증가하여 168억...,"루닛은 볼파라 인수를 통해 3분기 매출이 크게 증가했습니다. 구체적으로, 총 매출액...",다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,볼파라 인수를 통해 루닛의 3분기 매출이 전년 동기 대비 얼마나 증가했는지 서술하시오.
913,한화솔루션의 목표주가 하향 조정은 정치적 불확실성으로 인한 2025년 실적 전망 하...,한화솔루션은 목표주가를 기존 2.8만원에서 2.3만원으로 하향 조정하였습니다. 이 ...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,한화솔루션의 목표주가가 2.8만원에서 2.3만원으로 하향 조정된 주요 요인은 무엇인가?
914,삼성전자의 4분기 2024 매출액은 75.0조원으로 전 분기 대비 5.3% 감소하였...,삼성전자의 4분기 2024 잠정 실적에 따르면 매출액은 75.0조원으로 전 분기 대...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,"삼성전자의 4분기 2024 잠정 실적에서 매출액과 영업이익은 각각 얼마이며, DS ..."
915,앱 수수료가 17%로 인하될 경우 위메이드의 2024년도 지급수수료 절감액은 68...,위메이드는 모바일 매출 비중이 79%로 높아 앱 수수료 인하의 영향을 크게 받을 것...,다음 Context를 바탕으로 문맥과 수치를 참고하여 문제 해결 과정을 거쳐 정답을...,앱 수수료가 17%로 인하될 경우 위메이드의 2024년도 지급수수료 절감액은 얼마...


In [99]:
# 전처리 적용
cleaned_dataset = dataset.map(clean_text)

Map:   0%|          | 0/917 [00:00<?, ? examples/s]


TypeError: expected string or bytes-like object